# SQLite Search Utilities (`ouseful_sqlite_search_utils`)

Custom functions providing additional search utilities over text documents in SQLite databases.

Functions are defined that return:

- scalar values (`get_fragment()`m `get_longest_common_substring()`);
- aggregations (`join_sentences()`);
- tables (`get_sentences()`, `get_paragraphs()`, `re_search()`, `regex_search(`, `find_near_matches_all()`, `find_fuzzy_matches(`)).

In [1]:
#%pip install --upgrade git+https://github.com/innovationOUtside/ouseful-sqlite-search-utils.git

In [2]:
from sqlite_utils import Database

db_name = "sqlite_search_demo.db"

# While developing the script, recreate database each time...
db = Database(db_name, recreate=True)

## Snippets functions

Functions that search deep into a text record and return snippets or chunked results.

In [3]:
from ouseful_sqlite_search_utils import snippets

snippets.register_snippets(db.conn)

### `get_sentences(start_index, end_index, text)`

This function will return one or more sentences, one sentence per row, starting index of 1, based on sentence count within the returned document.

In [4]:
db["sentences"].insert({"id":1, "txt": "This is sentence 1. This is sentence 2. This is sentence 3."})
db["sentences"].insert({"id":2, "txt": "This is sentence 4."})
db["sentences"].insert({"id":3, "txt": "This is sentence 5. This is sentence 6."})

<Table sentences (id, txt)>

In [5]:
# Return just the first sentence
for i in db.query("SELECT id, sentence FROM sentences, get_sentences(1, 1, sentences.txt)"):
    print(i)

{'id': 1, 'sentence': 'This is sentence 1.'}
{'id': 2, 'sentence': 'This is sentence 4.'}
{'id': 3, 'sentence': 'This is sentence 5.'}


In [6]:
# Return second and third sentences
for i in db.query("SELECT id, sentence FROM sentences, get_sentences(2, 3, sentences.txt)"):
    print(i)

{'id': 1, 'sentence': 'This is sentence 2.'}
{'id': 1, 'sentence': 'This is sentence 3.'}
{'id': 3, 'sentence': 'This is sentence 6.'}


In [7]:
# Get the last sentence
q = """
SELECT id, sentence
FROM sentences, get_sentences(-1, NULL, sentences.txt)
"""

for i in db.query(q):
    print(i)

{'id': 1, 'sentence': 'This is sentence 3.'}
{'id': 2, 'sentence': 'This is sentence 4.'}
{'id': 3, 'sentence': 'This is sentence 6.'}


## `join_sentences()`

This aggregation function will join two or more sentences returned in a table from `get_sentences()`.

In [8]:
# Return second and third sentences and join them together
q = """
SELECT id, join_sentences(sentence)
FROM sentences, get_sentences(2, 3, sentences.txt)
GROUP BY id
"""

for i in db.query(q):
    print(i)

{'id': 1, 'sentence_join(sentence)': 'This is sentence 2. This is sentence 3.'}
{'id': 3, 'sentence_join(sentence)': 'This is sentence 6.'}


In [9]:
def get_fragment(text, startend):
    """Return substring from a text based on start and end substrings delimited by ::."""
    startend = startend.split("::")
    if startend[0] not in text or startend[1] not in text:
        return

    start_idx = text.index(startend[0])
    end_idx = text.index(startend[1])
    
    if end_idx < start_idx:
        return

    return text[start_idx: end_idx+len(startend[1])]

db.conn.create_function("get_fragment", 2, get_fragment)

## `get_paragraphs()`

Split the text into paragraphs and return the specified range of paragraphs (initial index is 1).

In [10]:
db["paragraphs"].insert({"id":1, "txt": "This is sentence 1, para 1. This is sentence 2.\n\nThis is sentence 3, para 2."})
db["paragraphs"].insert({"id":2, "txt": "This is sentence 1, para 1."})
db["paragraphs"].insert({"id":3, "txt": "This is sentence 1, para 1.\n\nThis is sentence 1, para 2."})

<Table paragraphs (id, txt)>

In [11]:
# Return just the second paragraph
for i in db.query("SELECT id, paragraph FROM paragraphs, get_paragraphs(2, 2, paragraphs.txt)"):
    print(i)

{'id': 1, 'paragraph': 'This is sentence 3, para 2.'}
{'id': 3, 'paragraph': 'This is sentence 1, para 2.'}


## `get_fragment()`

Get a fragment between a start and end phrase separated by `::`.

Note - if the start or end phrase is not unique, the first recorded match will be used.

In [12]:
q = """
SELECT get_fragment("nce 2::sentence 1", txt)
FROM sentences
"""

for i in db.query("SELECT get_fragment('Start here. This is the end. To here.', 'Start::he') AS fragment"):
    print(i)

{'fragment': 'Start he'}


In [13]:
q = """
SELECT get_fragment("nce 2::sentence 1", txt)
FROM sentences
"""

for i in db.query(q):
    print(i)

{'get_fragment("nce 2::sentence 1", txt)': None}
{'get_fragment("nce 2::sentence 1", txt)': None}
{'get_fragment("nce 2::sentence 1", txt)': None}


### `get_longest_common_substring()`

Get the longest common substring across two records:

In [14]:
q = """
SELECT get_longest_common_substring('This is sentence 1.', 'This is sentence 4.') AS common
"""

for i in db.query(q):
    print(i)

{'common': 'This is sentence '}


In [15]:
q = """
SELECT txt1.id AS id1, txt2.id AS id2, get_longest_common_substring(txt1.txt, txt2.txt)
FROM sentences AS txt1, sentences AS txt2
WHERE txt1.id < txt2.id
"""

for i in db.query(q):
    print(i)

{'id1': 1, 'id2': 2, 'get_longest_common_substring(txt1.txt, txt2.txt)': 'This is sentence '}
{'id1': 1, 'id2': 3, 'get_longest_common_substring(txt1.txt, txt2.txt)': '. This is sentence '}
{'id1': 2, 'id2': 3, 'get_longest_common_substring(txt1.txt, txt2.txt)': 'This is sentence '}


## Partial Matches

As well as retrieving records based on matching search terms exactly against elements contained the database, we can also make use of fuzzy search functions to identify partially or fuzzily matched items.

In [16]:
from ouseful_sqlite_search_utils import partial

partial.register_partials(db.conn)

### `re_search()`

To start with, we can use a simple Python regualr expression matcher:

In [17]:
q = """
SELECT id, re.* FROM sentences, re_search('sentence \d+', sentences.txt) AS re
"""

for i in db.execute(q):
    print(i)

(1, 'sentence 1')
(1, 'sentence 2')
(1, 'sentence 3')
(2, 'sentence 4')
(3, 'sentence 5')
(3, 'sentence 6')


In [18]:
from pandas import read_sql

read_sql(q, db.conn)

,id,match
0,1,sentence 1
1,1,sentence 2
2,1,sentence 3
3,2,sentence 4
4,3,sentence 5
5,3,sentence 6


### `regex_search()`

The [`regex`](https://github.com/mrabarnett/mrab-regex) package offers a wider range of regular expression matching rules, including the ability [partially match](https://github.com/mrabarnett/mrab-regex#added-partial-matches-hg-issue-102) expressions. For example: 

In [19]:
db["partial"].insert({"id":1, "txt": "The bin eater is..."})
db["partial"].insert({"id":2, "txt": "The in eater is..."})
db["partial"].insert({"id":3, "txt": "The bin eater is..."})
db["partial"].insert({"id":4, "txt": "The bin eating tradition is..."})

<Table partial (id, txt)>

In [20]:
# Detect  particular number of single character errors
q = """
SELECT id, reg.* FROM partial, regex_search('(sin eater){e<=3}', partial.txt) AS reg
"""

for i in db.execute(q):
    print(i)

(1, 'e bin eater', 2, 13)
(2, 'he in eater', 1, 12)
(3, 'e bin eater', 2, 13)
(4, 'bin eatin', 4, 13)


In [21]:
# Detect  particular number of single character deletions
q = """
SELECT id, reg.* FROM partial, regex_search('(sin eater){d<=1}', partial.txt) AS reg
"""

for i in db.execute(q):
    print(i)

(1, 'in eater', 5, 13)
(2, 'in eater', 4, 12)
(3, 'in eater', 5, 13)


In [22]:
# Detect  particular number of single character substitutions
q = """
SELECT id, reg.* FROM partial, regex_search('(sin eater){s<=1}', partial.txt) AS reg
"""

for i in db.execute(q):
    print(i)

(1, 'bin eater', 4, 13)
(2, ' in eater', 3, 12)
(3, 'bin eater', 4, 13)


### `find_near_matches()`

The `find_near_matches()` function uses the `fuzzysearch.find_near_matches()` to partially match the query string against records that occur in the database. If there are multiple matches, only the first match is returned.

In [23]:
for i in db.query("SELECT id, txt, find_near_matches('bin eater', partial.txt) AS fuzzy FROM partial"):
    print(i)

{'id': 1, 'txt': 'The bin eater is...', 'fuzzy': 'bin eater'}
{'id': 2, 'txt': 'The in eater is...', 'fuzzy': 'in eater'}
{'id': 3, 'txt': 'The bin eater is...', 'fuzzy': 'bin eater'}
{'id': 4, 'txt': 'The bin eating tradition is...', 'fuzzy': 'bin eatin'}


### `find_near_matches_all()`

The `find_near_matches_all()` uses the `fuzzysearch.find_near_matches()` to partially match the query string against records that occur in the database. If there are multiple matches, they are all returned as a table, one result per row.

In [24]:
for i in db.query("SELECT * FROM partial, find_near_matches_all('sin eater', partial.txt, 2)"):
    print(i)

{'id': 1, 'txt': 'The bin eater is...', 'matched': 'bin eater', 'start': 4, 'end': 13, 'dist': 1}
{'id': 2, 'txt': 'The in eater is...', 'matched': ' in eater', 'start': 3, 'end': 12, 'dist': 1}
{'id': 3, 'txt': 'The bin eater is...', 'matched': 'bin eater', 'start': 4, 'end': 13, 'dist': 1}


### `find_fuzzy_matches()`

The `find_fuzzy_matches()` function uses the `FuzzyMatcher` function from the `spaczz` package that adds fuzzy matching to `spacy` models.

In [25]:
_q = f"""
SELECT fz.*
FROM partial, find_fuzzy_matches('sin eater', partial.txt) AS fz ;
"""

list(db.query(_q))

[{'matched': 'The bin eater is...',
  'start': 1,
  'end': 3,
  'fragment': 'bin eater',
  'ratio': 89},
 {'matched': 'The in eater is...',
  'start': 1,
  'end': 3,
  'fragment': 'in eater',
  'ratio': 94},
 {'matched': 'The bin eater is...',
  'start': 1,
  'end': 3,
  'fragment': 'bin eater',
  'ratio': 89}]

## Spell-Checking

We can also run a spell-checker against the database, and return records with typographical errors highlighted.

*This currently runs using an `en-US` language pack.*

In [4]:
from ouseful_sqlite_search_utils import spellcheck

spellcheck.register_partials(db.conn)

In [6]:
text = 'This sentence is fine. A sentence with a error in the Hitchhiker’s Guide tot he Galaxy'

for i in db.execute(f'SELECT * FROM typo_highlighter("{text}");'):
    print(i)

('This sentence is fine. A sentence with <span style="color:red">a</span> error in the Hitchhiker’s Guide tot he ...',)
('... with a error in the Hitchhiker’s Guide <span style="color:red">tot he</span> Galaxy',)


In [7]:
text = 'There is a apeling error here. This is okay. Theze a coupl spelling erros here.'

for i in db.execute(f'SELECT * FROM typo_highlighter("{text}");'):
    print(i)

('There is <span style="color:red">a</span> apeling error here. This is okay. Theze...',)
('There is a <span style="color:red">apeling</span> error here. This is okay. Theze a coupl...',)
('... is a apeling error here. This is okay. <span style="color:red">Theze</span> a coupl spelling erros here.',)
('...eling error here. This is okay. Theze a <span style="color:red">coupl</span> spelling erros here.',)
('...e. This is okay. Theze a coupl spelling <span style="color:red">erros</span> here.',)


In [8]:
db["typos"].insert({"id":1, "txt": "There is a apeling error here."})
db["typos"].insert({"id":2, "txt": "This is okay."})
db["typos"].insert({"id":3, "txt": "Theze a coupl spelling erros here."})

<Table typos (id, txt)>

In [9]:
for i in db.execute(f'SELECT * FROM typos, typo_highlighter(typos.txt);'):
    print(i)

(1, 'There is a apeling error here.', 'There is <span style="color:red">a</span> apeling error here.')
(1, 'There is a apeling error here.', 'There is a <span style="color:red">apeling</span> error here.')
(3, 'Theze a coupl spelling erros here.', '<span style="color:red">Theze</span> a coupl spelling erros here.')
(3, 'Theze a coupl spelling erros here.', 'Theze a <span style="color:red">coupl</span> spelling erros here.')
(3, 'Theze a coupl spelling erros here.', 'Theze a coupl spelling <span style="color:red">erros</span> here.')
